In [1]:
import sys
sys.path.append(sys.path[0].split('NewEP3Exp')[0] + 'NewEP3Exp/Code')
sys.path.append(sys.path[0].split('NewEP3Exp')[0] + 'NewEP3Exp/Code/Instruments/SpectrumInstruments')

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
# import SpectrumAWG as SI
import time
from numba import jit


In [2]:
SR = 625e6 # max samling rate
l = 200e-3*SR # a "long" ringdown might take 200 ms
samps = np.arange(int(l))
tstamps = samps/SR # We'd probably want to pass around timestamps instead of sample number since that'll be something we compute a lot

# lets try to do this with some actually realistic parameters

f0 = 83e6 # frequency corresponding to Delta = 0

fm1 = 2.423806731707168*1e6 # Hz
fm2 = 3.330978169785124*1e6 # Hz

eta = -50 # Hz
delta_1 = -0.5e6 # Hz
delta_2 = -0.735e6 # Hz
delta_blue = 3.5e6 # Hz

F1_1 = f0 - fm1 + eta/2 + delta_1
F2_1 = f0 - fm2 - eta/2 + delta_1
F3 = f0 + delta_blue


F1_2 = f0 - fm1 + eta/2 + delta_2
F2_2 = f0 - fm2 - eta/2 + delta_2

A1_1 = .23452
A2_1 = .52341
A3 = .315673

A1_2 = .12367
A2_2 = .35256


def func1(samp, SR, f1, f2, a1, a2):
    return a1*np.cos(f1*samps/SR) + a2*np.cos(f2*samps/SR)

@jit(nopython = False)
def func2(samp, SR, f1, f2, a1, a2):
    return a1*np.cos(f1*samps/SR) + a2*np.cos(f2*samps/SR)

@jit(nopython = False, parallel = True)
def func3(samp, SR, f1, f2, a1, a2):
    return a1*np.cos(f1*samps/SR) + a2*np.cos(f2*samps/SR)

@jit(nopython = False, parallel = True)
def func4(t, f1, f2, a1, a2):
    return a1*np.cos(f1*t) + a2*np.cos(f2*t)




In [3]:
%%timeit 

y = func1(samps, SR, F1_1, F1_2, A1_1, A1_2)

6.17 s ± 158 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit 

y = func2(samps, SR, F1_1, F1_2, A1_1, A1_2)

4.41 s ± 58.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit 

y = func3(samps, SR, F1_1, F1_2, A1_1, A1_2)

789 ms ± 16.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit 

y = func4(samps, F1_1, F1_2, A1_1, A1_2)

743 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
class pulse_writer():
    def __init(pAOM = 83e6, SR = 625e6):
        self.pAOM = pAOM # probe AOM frequency  
        self.SR = SR